In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import warnings
import sqlite3  
output_path='./output/'
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

In [2]:
# 基本設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format',lambda x: '%.2f' %x)
warnings.filterwarnings("ignore")

### Insert CPU 的資料

In [51]:
#讀取資料
df_cpu=pd.read_csv(output_path+'cpu.csv')
df_cpu.head()

,class,name,price,etl_date,model,brand,performance,base_clock,boost_clock
0,處理器 CPU,Intel Processor 300【2核/4緒】3.9GHz/6M/UHD710/46W,2990,2024-03-03,Intel Processor,NaN,2核/4緒,3.9,3.9
1,處理器 CPU,Intel i3-14100F【4核/8緒】3.5GHz(↑4.7GHz)/12M/無內顯/...,3990,2024-03-03,Intel i3-14100F,NaN,4核/8緒,3.5,4.7
2,處理器 CPU,Intel i3-14100【4核/8緒】3.5GHz(↑4.7GHz)/12M/UHD73...,4800,2024-03-03,Intel i3-14100,NaN,4核/8緒,3.5,4.7
3,處理器 CPU,Intel i5-14400F【10核/16緒】2.5GHz(↑4.7G)/20M/無內顯/...,6800,2024-03-03,Intel i5-14400F,NaN,10核/16緒,2.5,4.7
4,處理器 CPU,Intel i5-14400【10核/16緒】2.5GHz(↑4.7G)/20M/UHD73...,7650,2024-03-03,Intel i5-14400,NaN,10核/16緒,2.5,4.7


In [52]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
import sqlite3
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

table_name='cpu'
conn.execute(f'''DELETE FROM {table_name};
           ''')
conn.commit()
conn.close()  

In [117]:
# 創建一個空的list
multiple_records = []

# 遍历 DataFrame 中的每一行
for index, row in df_cpu.iterrows():
    # 从每一行中获取需要的值
    id=row['model']
    name=row['name']
    price = row['price']
    model = row['model']
    date = row['etl_date']
    brand= row['brand']
    performance=row['performance']
    base_clock =row['base_clock']
    boost_clock = row['boost_clock']
    multiple_records.append((id,name,price, model, date,brand,performance,base_clock,boost_clock))

# 使用多行数据列表创建新的 DataFrame
new_df = pd.DataFrame(multiple_records, columns=['id','name','price', 'model', 'etl_date','brand','performance','base_clock','boost_clock'])

# 显示新的 DataFrame
new_df.head(20)

,id,name,price,model,etl_date,brand,performance,base_clock,boost_clock
0,Intel Processor,Intel Processor 300【2核/4緒】3.9GHz/6M/UHD710/46W,2990,Intel Processor,2024-03-03,NaN,2核/4緒,3.9,3.9
1,Intel i3-14100F,Intel i3-14100F【4核/8緒】3.5GHz(↑4.7GHz)/12M/無內顯/...,3990,Intel i3-14100F,2024-03-03,NaN,4核/8緒,3.5,4.7
2,Intel i3-14100,Intel i3-14100【4核/8緒】3.5GHz(↑4.7GHz)/12M/UHD73...,4800,Intel i3-14100,2024-03-03,NaN,4核/8緒,3.5,4.7
3,Intel i5-14400F,Intel i5-14400F【10核/16緒】2.5GHz(↑4.7G)/20M/無內顯/...,6800,Intel i5-14400F,2024-03-03,NaN,10核/16緒,2.5,4.7
4,Intel i5-14400,Intel i5-14400【10核/16緒】2.5GHz(↑4.7G)/20M/UHD73...,7650,Intel i5-14400,2024-03-03,NaN,10核/16緒,2.5,4.7
5,Intel i5-14500,Intel i5-14500【14核/20緒】2.6GHz(↑5G)/24M/UHD770/...,7900,Intel i5-14500,2024-03-03,NaN,14核/20緒,2.6,5.0
6,Intel i5-14600KF,Intel i5-14600KF【14核/20緒】3.5G(↑5.3G)/24M/無內顯/無...,9600,Intel i5-14600KF,2024-03-03,NaN,14核/20緒,3.5,5.3
7,Intel i5-14600K,Intel i5-14600K【14核/20緒】3.5G(↑5.3G)/24M/UHD770...,10200,Intel i5-14600K,2024-03-03,NaN,14核/20緒,3.5,5.3
8,Intel i7-14700F,Intel i7-14700F【20核/28緒】2.1GHz(↑5.4G)/33M/無內顯/...,12200,Intel i7-14700F,2024-03-03,NaN,20核/28緒,2.1,5.4
9,Intel i7-14700,Intel i7-14700【20核/28緒】2.1GHz(↑5.4G)/33M/UHD77...,13200,Intel i7-14700,2024-03-03,NaN,20核/28緒,2.1,5.4


In [54]:
#寫入資料到table裡面
import sqlite3
db_name = 'data/sqlchain.db'
cursor = sqlite3.connect(db_name)  

# 将数据插入表格
cursor.executemany('INSERT INTO cpu (id,name,price,model,etl_date,brand,performance,base_clock,boost_clock) VALUES (?,?,?,?,?,?,?,?,?)', multiple_records)
cursor.commit()
conn.close()  

### Insert GPU 的資料

In [1]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
import sqlite3
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

table_name='gpu'
conn.execute(f'''DELETE FROM {table_name};
           ''')
conn.commit()
conn.close()  

In [1]:
#讀取資料
df_gpu=pd.read_csv(output_path+'gpu.csv')
df_gpu.head(50)

NameError: name 'pd' is not defined

In [48]:
# 創建一個空的list
multiple_records = []

# 遍历 DataFrame 中的每一行
for index, row in df_gpu.iterrows():
    # 从每一行中获取需要的值
    id=row['model']
    name=row['name']
    price = row['price']
    model = row['model']
    date = row['etl_date']
    brand= row['brand']
    memory=row['memory']
    memory_value=row['memory_value']
    memory_unit=row['memory_unit']
    corefrequency_value=row['corefrequency_value']
    corefrequency_unit=row['corefrequency_unit']
    DDR=row['ddr']
    multiple_records.append((id,name,price, model, date,brand,memory,memory_value,memory_unit,corefrequency_value,corefrequency_unit,DDR))

In [49]:
#寫入資料到table裡面
import sqlite3
db_name = 'data/sqlchain.db'
cursor = sqlite3.connect(db_name)  

# 将数据插入表格
cursor.executemany('INSERT INTO gpu (id,name,price,model,etl_date,brand,memory,memory_value,memory_unit,corefrequency_value,corefrequency_unit,DDR) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)', multiple_records)
cursor.commit()
conn.close()  

### Insert 硬碟 的資料

In [88]:
#讀取資料
df_hdd=pd.read_csv(output_path+'hdd.csv')
df_hdd.head()

,class,name,price,etl_date,brand,capacity,drive_type,read_speed,write_speed,model,id
0,固態硬碟 M.2｜SSD,UMAX S330 240GB /2.5吋 讀:520MB寫:450MB/3D NAND F...,490,2024-03-03,UMAX S,240GB,SSD,520.0,450.0,UMAX S330,UMAX S330
1,固態硬碟 M.2｜SSD,UMAX S330 480GB /2.5吋 讀:560MB寫:450MB/3D NAND F...,880,2024-03-03,UMAX S,480GB,SSD,560.0,450.0,UMAX S330,UMAX S330
2,固態硬碟 M.2｜SSD,UMAX S330 960GB /2.5吋 讀:560MB寫:500MB/3D NAND F...,1600,2024-03-03,UMAX S,960GB,SSD,560.0,500.0,UMAX S330,UMAX S330
3,固態硬碟 M.2｜SSD,威剛 Ultimate SU650 120G/2.5吋/讀:520M/寫:320M/TLC顆...,350,2024-03-03,威剛 Ultimate SU,NaN,SSD,520.0,320.0,Ultimate SU650,Ultimate SU650
4,固態硬碟 M.2｜SSD,威剛 Ultimate SU650 240G/2.5吋/讀:520M/寫:450M/TLC顆...,599,2024-03-03,威剛 Ultimate SU,NaN,SSD,520.0,450.0,Ultimate SU650,Ultimate SU650


In [89]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
import sqlite3
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

table_name='hdd'
conn.execute(f'''DELETE FROM {table_name};
           ''')
conn.commit()
conn.close()  

In [90]:
# 創建一個空的list
multiple_records = []

# 遍历 DataFrame 中的每一行
for index, row in df_hdd.iterrows():
    # 从每一行中获取需要的值
    id=row['id']
    name=row['name']
    price = row['price']
    model = row['model']
    date = row['etl_date']
    brand= row['brand']
    capacity=row['capacity']
    read_speed=row['read_speed']
    write_speed=row['write_speed']
    drive_type=row['drive_type']
    multiple_records.append((id,name,price, model, date,brand,capacity,read_speed,write_speed,drive_type))

# 使用多行数据列表创建新的 DataFrame
new_df = pd.DataFrame(multiple_records, columns=['id','name','price', 'model', 'etl_date','brand','capacity','read_speed','write_speed','drive_type'])

# 显示新的 DataFrame
new_df.head()

,id,name,price,model,etl_date,brand,capacity,read_speed,write_speed,drive_type
0,UMAX S330,UMAX S330 240GB /2.5吋 讀:520MB寫:450MB/3D NAND F...,490,UMAX S330,2024-03-03,UMAX S,240GB,520.0,450.0,SSD
1,UMAX S330,UMAX S330 480GB /2.5吋 讀:560MB寫:450MB/3D NAND F...,880,UMAX S330,2024-03-03,UMAX S,480GB,560.0,450.0,SSD
2,UMAX S330,UMAX S330 960GB /2.5吋 讀:560MB寫:500MB/3D NAND F...,1600,UMAX S330,2024-03-03,UMAX S,960GB,560.0,500.0,SSD
3,Ultimate SU650,威剛 Ultimate SU650 120G/2.5吋/讀:520M/寫:320M/TLC顆...,350,Ultimate SU650,2024-03-03,威剛 Ultimate SU,NaN,520.0,320.0,SSD
4,Ultimate SU650,威剛 Ultimate SU650 240G/2.5吋/讀:520M/寫:450M/TLC顆...,599,Ultimate SU650,2024-03-03,威剛 Ultimate SU,NaN,520.0,450.0,SSD


In [91]:
#寫入資料到table裡面
import sqlite3
db_name = 'data/sqlchain.db'
cursor = sqlite3.connect(db_name)  

# 将数据插入表格
cursor.executemany('INSERT INTO hdd (id,name,price,model,etl_date,brand,CAPACITY,read_speed,write_speed,drive_type) VALUES (?,?,?,?,?,?,?,?,?,?)', multiple_records)
cursor.commit()
conn.close()  

### Insert 散熱器 的資料

In [4]:
#讀取資料
import pandas as pd
output_path='./output/'
df_cool=pd.read_csv(output_path+'cool.csv')
df_cool.head(50)

,class,name,price,etl_date,brand,tdp,rpm,type,model
0,散熱器｜散熱墊｜散熱膏,利民 M.2 2280 TYPE A B SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,199,2024-03-17,利民,Unknown,Unknown,Unknown,利民 M.2 2280 TYPE A B SSD 固態硬碟散熱片/鋁合金/單雙面皆適用
1,散熱器｜散熱墊｜散熱膏,利民 M.2 2280 SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,350,2024-03-17,利民,Unknown,Unknown,Unknown,利民 M.2 2280 SSD 固態硬碟散熱片/鋁合金/單雙面皆適用
2,散熱器｜散熱墊｜散熱膏,利民 M.2 2280 PRO SSD 固態硬碟散熱片/鋁合金+8 mm純銅導管/單雙面皆適用,400,2024-03-17,利民,Unknown,Unknown,Unknown,利民 M.2 2280 PRO SSD 固態硬碟散熱片/鋁合金+8 mm純銅導管/單雙面皆適用
3,散熱器｜散熱墊｜散熱膏,利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用,550,2024-03-17,利民,Unknown,Unknown,Unknown,利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用
4,散熱器｜散熱墊｜散熱膏,利民 HR-10 2280 PRO SSD 固態硬碟散熱器/4導管/3CM PWM風扇/單雙...,690,2024-03-17,利民,Unknown,Unknown,氣冷式,利民 HR-10 2280 PRO SSD 固態硬碟散熱器/4導管/3CM PWM風扇/單雙...
5,散熱器｜散熱墊｜散熱膏,快睿 Frostbit M.2 散熱片/2280/雙導管/可調式散熱鰭片,790,2024-03-17,快睿,Unknown,Unknown,Unknown,快睿 Frostbit M.2 散熱片/2280/雙導管/可調式散熱鰭片
6,散熱器｜散熱墊｜散熱膏,Montech Air Cooler 210 6導管/高度15.3cm/全黑化/A.RGB/...,1090,2024-03-17,Montech,210W,Unknown,Unknown,Montech Air Cooler 210 6導管/高度15.3cm/全黑化/A.RGB/...
7,散熱器｜散熱墊｜散熱膏,Montech Metal DT24 Base 6導管/高度15.3/雙塔雙扇/TDP:27...,1590,2024-03-17,Montech,270W,Unknown,Unknown,Montech Metal DT24 Base 6導管/高度15.3/雙塔雙扇/TDP:27...
8,散熱器｜散熱墊｜散熱膏,Montech Metal DT24 Premium 6導管/高15.8/雙塔雙扇/全黑化/...,1890,2024-03-17,Montech,Unknown,Unknown,Unknown,Montech Metal DT24 Premium 6導管/高15.8/雙塔雙扇/全黑化/...
9,散熱器｜散熱墊｜散熱膏,ID-COOLING IS-40X 散熱器 下吹式/4導管/高4.5/全黑化/TDP:100...,790,2024-03-17,ID-COOLING,100W,Unknown,Unknown,ID-COOLING IS-40X 散熱器 下吹式/4導管/高4.5/全黑化/TDP:100...


In [14]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
import sqlite3
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

table_name='cool'
conn.execute(f'''DELETE FROM {table_name};
           ''')
conn.commit()
conn.close()  

In [15]:
# 創建一個空的list
multiple_records = []

# 遍历 DataFrame 中的每一行
for index, row in df_cool.iterrows():
    # 从每一行中获取需要的值
    id=row['model']
    name=row['name']
    price = row['price']
    model = row['model']
    date = row['etl_date']
    brand= row['brand']
    multiple_records.append((id,name,price, model, date,brand))

# 使用多行数据列表创建新的 DataFrame
new_df = pd.DataFrame(multiple_records, columns=['id','name','price', 'model', 'etl_date','brand'])

# 显示新的 DataFrame
new_df.head()

,id,name,price,model,etl_date,brand
0,利民 M.2 2280 TYPE A B SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,利民 M.2 2280 TYPE A B SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,199,利民 M.2 2280 TYPE A B SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,2024-03-03,利民
1,利民 M.2 2280 SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,利民 M.2 2280 SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,350,利民 M.2 2280 SSD 固態硬碟散熱片/鋁合金/單雙面皆適用,2024-03-03,利民
2,利民 M.2 2280 PRO SSD 固態硬碟散熱片/鋁合金+8 mm純銅導管/單雙面皆適用,利民 M.2 2280 PRO SSD 固態硬碟散熱片/鋁合金+8 mm純銅導管/單雙面皆適用,400,利民 M.2 2280 PRO SSD 固態硬碟散熱片/鋁合金+8 mm純銅導管/單雙面皆適用,2024-03-03,利民
3,利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用,利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用,550,利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用,2024-03-03,利民
4,利民 HR-10 2280 PRO SSD 固態硬碟散熱器/4導管/3CM PWM風扇/單雙...,利民 HR-10 2280 PRO SSD 固態硬碟散熱器/4導管/3CM PWM風扇/單雙...,690,利民 HR-10 2280 PRO SSD 固態硬碟散熱器/4導管/3CM PWM風扇/單雙...,2024-03-03,利民


In [16]:
#寫入資料到table裡面
import sqlite3
db_name = 'data/sqlchain.db'
cursor = sqlite3.connect(db_name)  

# 将数据插入表格
cursor.executemany('INSERT INTO cool (id,name,price,model,etl_date,brand) VALUES (?,?,?,?,?,?)', multiple_records)
cursor.commit()
conn.close()  

### Insert 記憶體 的資料

In [78]:
#讀取資料
df_ram=pd.read_csv(output_path+'ram.csv')
df_ram.head()

,class,name,price,etl_date,brand,module_Spec,capacity,capacity_value,capacity_unit
0,記憶體 RAM,UMAX 單條8GB DDR5-5600/CL46【具XMP、EXPO參數】,749,2024-03-03,UMAX,DDR5,8GB,8,GB
1,記憶體 RAM,UMAX 單條16GB DDR5-5600/CL46【具XMP、EXPO參數】,1299,2024-03-03,UMAX,DDR5,16GB,16,GB
2,記憶體 RAM,UMAX 單條32GB DDR5-5600/CL46【具XMP、EXPO參數】,2599,2024-03-03,UMAX,DDR5,32GB,32,GB
3,記憶體 RAM,金士頓 單條16GB DDR5-5600(CL36) FURY Beast (獸獵者)【具X...,1580,2024-03-03,金士頓,DDR5,16GB,16,GB
4,記憶體 RAM,金士頓 單條16GB DDR5-5600(CL36) FURY Beast RGB (獸獵者...,1750,2024-03-03,金士頓,DDR5,16GB,16,GB


In [79]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
import sqlite3
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

table_name='ram'
conn.execute(f'''DELETE FROM {table_name};
           ''')
conn.commit()
conn.close()  

In [80]:
# 創建一個空的list
multiple_records = []

# 遍历 DataFrame 中的每一行
for index, row in df_ram.iterrows():
    # 从每一行中获取需要的值
    id=row['name']
    model=row['name']
    name=row['name']
    price = row['price']
    date = row['etl_date']
    brand= row['brand']
    module_spec=row['module_Spec']
    capacity=row['capacity']
    capacity_value=row['capacity_value']
    capacity_unit=row['capacity_unit']
    multiple_records.append((id,model,name,price, date,brand,module_spec,capacity,capacity_value,capacity_unit))

# 使用多行数据列表创建新的 DataFrame
new_df = pd.DataFrame(multiple_records, columns=['id','model','name','price', 'etl_date','brand','module_spec','capacity','capacity_value','capacity_unit'])

# 显示新的 DataFrame
new_df.head()

,id,model,name,price,etl_date,brand,module_spec,capacity,capacity_value,capacity_unit
0,UMAX 單條8GB DDR5-5600/CL46【具XMP、EXPO參數】,UMAX 單條8GB DDR5-5600/CL46【具XMP、EXPO參數】,UMAX 單條8GB DDR5-5600/CL46【具XMP、EXPO參數】,749,2024-03-03,UMAX,DDR5,8GB,8,GB
1,UMAX 單條16GB DDR5-5600/CL46【具XMP、EXPO參數】,UMAX 單條16GB DDR5-5600/CL46【具XMP、EXPO參數】,UMAX 單條16GB DDR5-5600/CL46【具XMP、EXPO參數】,1299,2024-03-03,UMAX,DDR5,16GB,16,GB
2,UMAX 單條32GB DDR5-5600/CL46【具XMP、EXPO參數】,UMAX 單條32GB DDR5-5600/CL46【具XMP、EXPO參數】,UMAX 單條32GB DDR5-5600/CL46【具XMP、EXPO參數】,2599,2024-03-03,UMAX,DDR5,32GB,32,GB
3,金士頓 單條16GB DDR5-5600(CL36) FURY Beast (獸獵者)【具X...,金士頓 單條16GB DDR5-5600(CL36) FURY Beast (獸獵者)【具X...,金士頓 單條16GB DDR5-5600(CL36) FURY Beast (獸獵者)【具X...,1580,2024-03-03,金士頓,DDR5,16GB,16,GB
4,金士頓 單條16GB DDR5-5600(CL36) FURY Beast RGB (獸獵者...,金士頓 單條16GB DDR5-5600(CL36) FURY Beast RGB (獸獵者...,金士頓 單條16GB DDR5-5600(CL36) FURY Beast RGB (獸獵者...,1750,2024-03-03,金士頓,DDR5,16GB,16,GB


In [82]:
#寫入資料到table裡面
import sqlite3
db_name = 'data/sqlchain.db'
cursor = sqlite3.connect(db_name)  

# 将数据插入表格
cursor.executemany('INSERT INTO ram (id,model,name,price,etl_date,brand,module_spec,capacity,capacity_value,capacity_unit) VALUES (?,?,?,?,?,?,?,?,?,?)', multiple_records)
cursor.commit()
conn.close()  

### Insert 主機板 的資料

In [55]:
df_motherboard=pd.read_csv(output_path+'motherboard.csv')
df_motherboard.head()

,class,name,price,etl_date,model,size,brand,company
0,主機板 MB,華碩 Tinker Fanless Aluminum Case 樹莓派鋁質機殼 (2/2S ...,399,2024-03-03,Unknown,Unknown,華碩,華碩
1,主機板 MB,華碩 PRO WS W680M-ACE SE(M-ATX/1A1H1P/雙Intel 2.5...,12990,2024-03-03,M-ACE,ATX,華碩,華碩
2,主機板 MB,華碩 PRO WS W680-ACE(ATX/1A1H1P/雙Intel 2.5Gb/註四年...,9990,2024-03-03,Unknown,ATX,華碩,華碩
3,主機板 MB,華碩 Pro WS W790E-SAGE SE(EEB/8*DDR5/2*Intel 10G...,39990,2024-03-03,E-SAGE,Unknown,華碩,華碩
4,主機板 MB,華擎 W790 WS(EEB/8*DDR5/2*Marvell 10G+Intel 2.5G...,34990,2024-03-03,Unknown,Unknown,Unknown,Unknown


In [83]:
# 清空資料(這邊是為了避免資料因為爬蟲出現重複，所以先清空table全部資料)
import sqlite3
db_name = 'data/sqlchain.db'
conn = sqlite3.connect(db_name)  

table_name='mainboard'
conn.execute(f'''DELETE FROM {table_name};
           ''')
conn.commit()
conn.close()  

In [84]:
# 創建一個空的list
multiple_records = []

# 遍历 DataFrame 中的每一行
for index, row in df_motherboard.iterrows():
    # 从每一行中获取需要的值
    name=row['name']
    model=row['model']
    price = row['price']
    date = row['etl_date']
    brand= row['brand']
    multiple_records.append((name,price,model ,date,brand))

In [85]:
#寫入資料到table裡面
import sqlite3
db_name = 'data/sqlchain.db'
cursor = sqlite3.connect(db_name)  

# 将数据插入表格
cursor.executemany('INSERT INTO mainboard (model,name,price,etl_date,brand) VALUES (?,?,?,?,?)', multiple_records)
cursor.commit()
conn.close()  

### 查詢資料庫

In [97]:
### 驗證查詢
conn = sqlite3.connect(db_name)  
sql1=f'''
select *
from gpu
'''
test=pd.read_sql(sql1,conn)
conn.close()
test.head()

,ID,NAME,PRICE,MODEL,BRAND,ETL_DATE,memory,memory_value,memory_unit,corefrequency_value,corefrequency_unit,DDR
0,Unknown,INTEL 原廠卡 ARC A750 8G(2050MHz/27cm/三年保)限量版顯示卡,5490,Unknown,Unknown,2024-03-03,8G,8,G,2050,MHz,NaN
1,Unknown,INTEL 原廠卡 ARC A750 8G(2050MHz/27cm/三年保)限量版顯示卡-...,4990,Unknown,Unknown,2024-03-03,8G,8,G,2050,MHz,NaN
2,Unknown,ACER 宏�� Predator Arc A750 8G OC(2200MHz/27cm/...,5900,Unknown,Unknown,2024-03-03,8G,8,G,2200,MHz,NaN
3,Unknown,ACER 宏�� Predator Arc A770 16G(2400MHz/27cm/三年...,11590,Unknown,Unknown,2024-03-03,16G,16,G,2400,MHz,NaN
4,Unknown,ACER 宏�� Predator Arc A770 16G(2400MHz/27cm/三年...,9990,Unknown,Unknown,2024-03-03,16G,16,G,2400,MHz,NaN


In [114]:
test.head(30)

,ID,NAME,PRICE,MODEL,BRAND,ETL_DATE,memory,memory_value,memory_unit,corefrequency_value,corefrequency_unit,DDR
0,Unknown,INTEL 原廠卡 ARC A750 8G(2050MHz/27cm/三年保)限量版顯示卡,5490,Unknown,Unknown,2024-03-03,8G,8,G,2050,MHz,NaN
1,Unknown,INTEL 原廠卡 ARC A750 8G(2050MHz/27cm/三年保)限量版顯示卡-...,4990,Unknown,Unknown,2024-03-03,8G,8,G,2050,MHz,NaN
2,Unknown,ACER 宏�� Predator Arc A750 8G OC(2200MHz/27cm/...,5900,Unknown,Unknown,2024-03-03,8G,8,G,2200,MHz,NaN
3,Unknown,ACER 宏�� Predator Arc A770 16G(2400MHz/27cm/三年...,11590,Unknown,Unknown,2024-03-03,16G,16,G,2400,MHz,NaN
4,Unknown,ACER 宏�� Predator Arc A770 16G(2400MHz/27cm/三年...,9990,Unknown,Unknown,2024-03-03,16G,16,G,2400,MHz,NaN
5,Unknown,華擎 A770 Phantom Gaming 16G OC(2200MHz/31cm/三風扇...,10990,Unknown,Unknown,2024-03-03,16G,16,G,2200,MHz,NaN
6,Unknown,微星 N210-MD1G/D3(589MHz/1G DDR3/風扇版/14.5cm/三年保),1250,Unknown,微星,2024-03-03,1G,1,G,589,MHz,3.0
7,GT710,華碩 GT710-SL-2GD3-BRK-EVO(954MHz/2G DDR3/17cm/註...,1690,GT710,華碩,2024-03-03,2G,2,G,954,MHz,3.0
8,GT710,華碩 GT710-SL-2GD5-BRK-EVO(954MHz/2G DDR5/17cm/註...,1790,GT710,華碩,2024-03-03,2G,2,G,954,MHz,5.0
9,Unknown,技嘉 N710D3-2GL(954MHz/2G DDR3/14.4cm/三年保),1790,Unknown,技嘉,2024-03-03,2G,2,G,954,MHz,3.0
